## Expectation Reflection + Least Absolute Deviations

In the following, we demonstrate how to apply Least Absolute Deviations (LAD) for classification task such as medical diagnosis.

We import the necessary packages to the Jupyter notebook:

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,KFold
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,\
recall_score,roc_curve,auc

from scipy import linalg
from scipy.special import erf as sperf

import expectation_reflection as ER

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from function import split_train_test,make_data_balance

In [2]:
np.random.seed(1)

First of all, the processed data are imported.

In [3]:
data_list = np.loadtxt('data_list.txt',dtype='str')
#data_list = ['29parkinson','30paradox2','31renal','32patientcare','33svr','34newt','35pcos']
print(data_list)

['1paradox' '2peptide' '3stigma' '4nki' '5mental' '6smoking' '7anemia'
 '8language' '9coag' '10tazamia' '11hepato' '12heat' '13ef' '14cervix'
 '15heart' '16liver' '17nwosu' '18school' '19ibs' '21survival' '101kidney'
 '102breast_cancer' '103diabetes_niddk' '104diabetic_retinopathy'
 '29parkinson' '30paradox2' '31renal' '33svr' '35pcos' '36probiotic']


In [4]:
def read_data(data_id):    
    data_name = data_list[data_id]
    print('data_name:',data_name)
    Xy = np.loadtxt('../data/%s/data_processed.dat'%data_name) 
    X = Xy[:,:-1]
    y = Xy[:,-1]

    #print(np.unique(y,return_counts=True))

    X,y = make_data_balance(X,y)

    print(np.unique(y,return_counts=True))

    X, y = shuffle(X, y, random_state=1)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state = 1)
    
    sc = MinMaxScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train,X_test,y_train,y_test

In [5]:
def infer_LAD(x, y, regu=0.1,tol=1e-6, max_iter=1000):
## 2019.12.26: Jungmin's code    
    weights_limit = sperf(1e-10)*1e10
    
    s_sample, s_pred = x.shape
    s_sample, s_target = y.shape
    
    mu = np.zeros(x.shape[1])

    w_sol = 0.0*(np.random.rand(s_pred,s_target) - 0.5)
    b_sol = np.random.rand(1,s_target) - 0.5

#     print(weights.shape)
    for index in range(s_target):
        error, old_error = np.inf, 0
        weights = np.ones((s_sample, 1))
        cov = np.cov(np.hstack((x,y[:,index][:,None])), rowvar=False, \
                     ddof=0, aweights=weights.reshape(s_sample))
        cov_xx, cov_xy = cov[:s_pred,:s_pred],cov[:s_pred,s_pred:(s_pred+1)]

#         print(cov.shape, cov_xx.shape, cov_xy.shape)
        counter = 0
        while np.abs(error-old_error) > tol and counter < max_iter:
            counter += 1
            old_error = np.mean(np.abs(b_sol[0,index] + x.dot(w_sol[:,index]) - y[:,index]))
#             old_error = np.mean(np.abs(b_sol[0,index] + x_test.dot(w_sol[:,index]) - y_test[:,index]))
#             print(w_sol[:,index].shape, npl.solve(cov_xx, cov_xy).reshape(s_pred).shape)

            # 2019.12.26: Tai - added regularization
            sigma_w = np.std(w_sol[:,index])
                
            w_eq_0 = np.abs(w_sol[:,index]) < 1e-10
            mu[w_eq_0] = 2./np.sqrt(np.pi)
        
            mu[~w_eq_0] = sigma_w*sperf(w_sol[:,index][~w_eq_0]/sigma_w)/w_sol[:,index][~w_eq_0]
                                                        
            w_sol[:,index] = np.linalg.solve(cov_xx + regu*np.diag(mu),cov_xy).reshape(s_pred)
        
            b_sol[0,index] = np.mean(y[:,index]-x.dot(w_sol[:,index]))
            weights = (b_sol[0,index] + x.dot(w_sol[:,index]) - y[:,index])
            sigma = np.std(weights)
            error = np.mean(np.abs(weights))
#             error = np.mean(np.abs(b_sol[0,index] + x_test.dot(w_sol[:,index]) - y_test[:,index]))
            weights_eq_0 = np.abs(weights) < 1e-10
            weights[weights_eq_0] = weights_limit
            weights[~weights_eq_0] = sigma*sperf(weights[~weights_eq_0]/sigma)/weights[~weights_eq_0]
            
            #weights = sigma*sperf(weights/sigma)/weights            
            cov = np.cov(np.hstack((x,y[:,index][:,None])), rowvar=False, \
                         ddof=0, aweights=weights.reshape(s_sample))
            cov_xx, cov_xy = cov[:s_pred,:s_pred],cov[:s_pred,s_pred:(s_pred+1)]
#             print(old_error,error)
    #return b_sol,w_sol 
    return b_sol[0][0],w_sol[:,0] # for only one target case

In [6]:
def fit_LAD(x,y,niter_max,l2):      
    # convert 0, 1 to -1, 1
    y1 = 2*y - 1.
   
    #print(niter_max)    
    n = x.shape[1]
    
    x_av = np.mean(x,axis=0)
    dx = x - x_av
    c = np.cov(dx,rowvar=False,bias=True)

    # 2019.07.16:  
    c += l2*np.identity(n) / (2*len(y))
    c_inv = linalg.pinvh(c)

    # initial values
    h0 = 0.
    w = np.random.normal(0.0,1./np.sqrt(n),size=(n))
    
    cost = np.full(niter_max,100.)
    for iloop in range(niter_max):
        h = h0 + x.dot(w)
        y1_model = np.tanh(h/2.)    

        # stopping criterion
        p = 1/(1+np.exp(-h))                
        cost[iloop] = ((p-y)**2).mean()

        #h_test = h0 + x_test.dot(w)
        #p_test = 1/(1+np.exp(-h_test))
        #cost[iloop] = ((p_test-y_test)**2).mean()

        if iloop>0 and cost[iloop] >= cost[iloop-1]: break

        # update local field
        t = h!=0    
        h[t] *= y1[t]/y1_model[t]
        h[~t] = 2*y1[~t]

        # find w from h    
        #h_av = h.mean()
        #dh = h - h_av 
        #dhdx = dh[:,np.newaxis]*dx[:,:]
        #dhdx_av = dhdx.mean(axis=0)
        #w = c_inv.dot(dhdx_av)
        #h0 = h_av - x_av.dot(w)

        # 2019.12.26: 
        h0,w = infer_LAD(x,h[:,np.newaxis],l2)

    return h0,w

In [7]:
def measure_performance(X_train,X_test,y_train,y_test):

    n = X_train.shape[1]

    l2 = [0.0001,0.001,0.01,0.1,1.,10.,100.]
    #l2 = [0.0001,0.001,0.01,0.1,1.,10.]
    nl2 = len(l2)

    # cross validation 
    kf = 4   
    kfold = KFold(n_splits=kf,shuffle=False,random_state=1)

    h01 = np.zeros(kf)
    w1 = np.zeros((kf,n))
    cost1 = np.zeros(kf)

    h0 = np.zeros(nl2)
    w = np.zeros((nl2,n))
    cost = np.zeros(nl2)            
    for il2 in range(len(l2)):            
        for i,(train_index,val_index) in enumerate(kfold.split(y_train)):
            X_train1, X_val = X_train[train_index], X_train[val_index]
            y_train1, y_val = y_train[train_index], y_train[val_index]
            #h01[i],w1[i,:] = ER.fit(X_train1,y_train1,niter_max=100,l2=l2[il2])
            #h01[i],w1[i,:] = ER.fit_LAD(X_train1,y_train1,niter_max=100,l2=l2[il2])
            h01[i],w1[i,:] = fit_LAD(X_train1,y_train1,niter_max=100,l2=l2[il2])

            y_val_pred,p_val_pred = ER.predict(X_val,h01[i],w1[i])
            cost1[i] = ((p_val_pred - y_val)**2).mean()

        h0[il2] = h01.mean(axis=0)
        w[il2,:] = w1.mean(axis=0)
        cost[il2] = cost1.mean()

    # optimal value of l2:
    il2_opt = np.argmin(cost)
    print('optimal l2:',l2[il2_opt])

    # performance:
    y_test_pred,p_test_pred = ER.predict(X_test,h0[il2_opt],w[il2_opt,:])

    fp,tp,thresholds = roc_curve(y_test, p_test_pred, drop_intermediate=False)

    roc_auc = auc(fp,tp)
    #print('AUC:', roc_auc)

    acc = accuracy_score(y_test,y_test_pred)
    #print('Accuracy:', acc)

    precision = precision_score(y_test,y_test_pred)
    #print('Precision:',precision)

    recall = recall_score(y_test,y_test_pred)
    #print('Recall:',recall)

    return acc,roc_auc,precision,recall

In [8]:
n_data = len(data_list)
roc_auc = np.zeros(n_data)   ; acc = np.zeros(n_data)
precision = np.zeros(n_data) ; recall = np.zeros(n_data)
for data_id in range(n_data):
    X_train,X_test,y_train,y_test = read_data(data_id)
    acc[data_id],roc_auc[data_id],precision[data_id],recall[data_id] =\
            measure_performance(X_train,X_test,y_train,y_test)
    print(data_id,acc[data_id],roc_auc[data_id]) 

data_name: 1paradox
(array([0., 1.]), array([60, 60]))
optimal l2: 0.001
0 0.8166666666666667 0.8736780258519389
data_name: 2peptide
(array([0., 1.]), array([23, 23]))
optimal l2: 0.0001
1 0.9130434782608695 1.0
data_name: 3stigma
(array([0., 1.]), array([2725, 2725]))
optimal l2: 0.001
2 0.9922935779816514 0.9926971007107359
data_name: 4nki
(array([0., 1.]), array([77, 77]))
optimal l2: 1.0
3 0.8311688311688312 0.8891891891891891
data_name: 5mental
(array([0., 1.]), array([147, 147]))
optimal l2: 1.0
4 0.6530612244897959 0.6975881261595548
data_name: 6smoking
(array([0., 1.]), array([722, 722]))
optimal l2: 0.001
5 1.0 1.0
data_name: 7anemia
(array([0., 1.]), array([43, 43]))
optimal l2: 0.001
6 0.6744186046511628 0.8031674208144797
data_name: 8language
(array([0., 1.]), array([267, 267]))
optimal l2: 0.01
7 0.7640449438202247 0.828972487366648
data_name: 9coag
(array([0., 1.]), array([504, 504]))
optimal l2: 0.001
8 0.5972222222222222 0.6531715388858247
data_name: 10tazamia
(array([0

In [9]:
print('acc_mean:',acc.mean())
print('roc_mean:',roc_auc.mean())

print('precision:',precision.mean())
print('recall:',recall.mean())

acc_mean: 0.8158540934067182
roc_mean: 0.8816093081200438
precision: 0.8381962239057751
recall: 0.8033483899691063


In [10]:
#np.savetxt('ER_LAD_result_30sets.dat',(roc_auc,acc,precision,recall),fmt='%f')